# Reversign delle API di Course Catalogue?

Sembra un poco surreale ma pare che Course Catalogue esponga vari endpoint per accedere ai dati... ossia ci sono già varie API pubbliche (che i sistemisti non conoscono?).

## 1. Accesso parametrico agli insegnamenti in Catalogo

Partendo da quanto indicato in GDA è emerso che è possibile accedere agli insegnamenti con degli indirizzi parametrici di cui sono forniti alcuni esempi a seguire.

Per l'insegnamento magistrale **217BB "Accelerator Physics"** abbiamo (parametri separati artificialmente... va concatenato tutto nell'URL)

`https://unipi.coursecatalogue.cineca.it/af/2024?`
* `corso=WFI-LM&`
* `annoOrdinamento=2023&`
* `ad=217BB`

Per l'insegnamento **288BB "Astrofisica Generale"** abbiamo invece 

`https://unipi.coursecatalogue.cineca.it/af/2024?`
* `corso=FIS-L&`
* `annoOrdinamento=2017&`
* `ad=288BB`

Ed esiste anche un modo per accedere alle diverse varianti di un insegnamento, come nel caso di **632AA "Analisi Matematica"**

`https://unipi.coursecatalogue.cineca.it/af/2024?`
* `corso=FIS-L&`
* `annoOrdinamento=2017&`
* `ad=632AA&`
* `domPart=A`

In conclusione c'è una sintassi precisa e bisogna conoscere semplicemente la giusta combinazione di codice dell'insegnamento e dell'anno dell'Ordinamento. I codici recuperati su GDA permettono di specificare degli ulteriori parametri (di fatto, guardando come risponde il sistema, opzionali) fra cui

* `pds` per indicare il piano di studio... qui vanno conosciuti i codici, che sono gli stessi che compaiono in GDA. Non sembra particolarmente utile ma cambia la modalità di visualizzazione dell'insegnamento sulla pagina web.
* `coorte` anno della coorte, questo potrebbe essere interessante... in assenza del parametro non mi è chiaro cosa sceglie il sistema
* `sede` tipicamente uguale a 1059 che credo significhi Università di Pisa
* `lingua` ... non chiaro perché indicarlo dato che è una proprietà dell'insegnamento
* `modDid` significato non chiaro e tipicamente di valore C... che mi pare di capire stia per "Convenzionale".

## 2. Digging deeper

Guardando il codice HTML a questi indirizzi non si capisce nulla, e il contenuto vero richiede Javascript per essere caricato. Secondo GPT questo avviene tramite la app Angular. Non è chiaro se sia vero ma usando tasto destro inspect e DevTools su Chrome e andando sul tab Network è possibile rintracciare la chiamate che permettono di popolare le pagine dei corsi... ossia - *surprise-surprise* - il sito di Course Catalogue ha già una inferfaccia API ed espone già un tot di endpoint.

Per esempio, è possibile ottenere un `JSON` con tutte le info dell'insegnamento con 

`https://unipi.coursecatalogue.cineca.it/api/v1/attivitaFormativa?`
* `aaOfferta=2024&`
* `corso=WFI-LM&`
* `annoOrdinamento=2023&`
* `ad=217BB`

Diversamente dal caso precedente, qui il catalogo fornisce **DIRETTAMENTE** i dati JSON dell'insegnamento, senza passare da Javascript e quindi con una richiesta web standard.

## 3. Il mistero dei codici interni Cineca

I link del Catalogo hanno tipicamente dei codici complessi la cui origine non è chiara. Per esempio i link web parametrici di cui sopra vengono reindirizzati su questi codici di orgine ignota. Anche questi tuttavia derivano da una interrogazione, evidentemente, e qui è stata rintracciata. D'altra parte se esiste un redirect, è possibile "chiedere" al catalogo quali siano questi codici.

Quando viene caricato l'URL..

`https://unipi.coursecatalogue.cineca.it/af/2024?corso=WFI-LM&annoOrdinamento=2023&ad=217BB`

questo viene reindirizzato a 

`https://unipi.coursecatalogue.cineca.it/insegnamenti/2024/52567_697863_71649/2023/52567/10452?annoOrdinamento=2023`

dove `52567_697863_71649` è un esempio del codice misterioso dell'insegnamento citato sopra, che non è chiaro come riagganciare al codice GDA `217BB`. Questo è evidentemente un codice interno al database Cineca. Ebbene... nel JSON di cui sopra il codice compare come valore della chiava `cod`.

In [5]:
# Esempio: come interrogare Course Catalogue per avere il codice interno di 217BB

import requests
import yaml

targetFile = "infoAF2.yaml"
url = "https://unipi.coursecatalogue.cineca.it/api/v1/attivitaFormativa"
params = {
    "aaOfferta": 2024,
    "corso": "WFI-LM",
    "annoOrdinamento": 2023,
    "ad": "323BB"
}
response = requests.get(url, params=params)

if response.status_code == 200:

    data = response.json()
    print(f"Titolo: {data['des_it']}")
    print(f"SSD:    {data['ssd']}")
    print(f"CFU:    {data['crediti']} CFU")
    docenti = ['     -> '+xx['des'] for xx in data['docenti']]
    print(f"Docenti:")
    print("\n".join(docenti))
    print(f"Codice interno: {data['cod']}")
    with open(targetFile, "w", encoding="utf-8") as file_yaml:
        yaml.dump(data, file_yaml, allow_unicode=True, sort_keys=False)
    
else:
    print(f"Errore nella richiesta: {response.status_code}")


Errore nella richiesta: 404


## 4. Altri endpoint esposti da Course Catalogue

A questo punto, ho esplorato con DevTools varie altre pagine... fra cui quella di ricerca degli insegnamenti. Ricerca fruttuosa perché sembra possibile fare una richiesta di tipo POST (con dei dati inviati al server) al seguente e nuovo endpoint

`https://unipi.coursecatalogue.cineca.it/api/v1/ricercaInsegnamenti`

Nel *payload* vanno indicati i parametri di ricerca nel formato di un dizionario JSON. Sotto un esempio di utilizzo. Si noti:

* diversamente dal form di ricerca web, qui è possibile specificare anche `None` nel parametro di anno di offerta `anno_off`, e ottenerne così una lista di TUTTE le risposte possibili di tutti gli anni.
* nel campo `insegnamento` si può inserire sia il codice che parti del titolo 
* `anno_imm` volendo seleziona la coorte
* `corsiSearchString` non è chiaro che significhi, non è esposto sul form di ricerca.
* La richiesta diretta all'API aggira alcuni dei limiti dei form online, per esempio è possibile richiedere TUTTI gli insegnamenti erogati presso il Dipartimento di Fisica nel 2024. Per indicare il dipartimento va usato il codice del dipartimento, che per fisica è 10184.

In [ ]:
# Interrogazione dell'API per ottenere informazioni sugli insegnamenti

import requests
import yaml

targetFile = "searchInsegnamenti.yaml"
url = "https://unipi.coursecatalogue.cineca.it/api/v1/ricercaInsegnamenti"
payload = {
    "insegnamento": "",
    "docente": None,
    "anno_imm": None,
    "anno_off": "2024",   
    "dipartimento": 10184,
    "ssdSearchString": None,
    "tipoCorso": None,
    "lingua": None,
    "corsiSearchString": None,
}
headers = {"Content-Type": "application/json"}
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    risultati = response.json()
    for tmp in risultati:
        print(f"[{tmp['adCod']}] {tmp['des_it']}")
        print(f"    Percorso:     {tmp['corso_percorso_des_it']}")
        print(f"    Codice:       {tmp['cod']}")
        print(f"    Coorte:       {tmp['corso_aa']}") 
        print(f"    Dipartimento: {tmp['dip_des_it']}")       
        print()
    with open(targetFile, "w", encoding="utf-8") as f:
        yaml.dump(risultati, f, allow_unicode=True, sort_keys=False)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)


[033BB] LABORATORIO 2 a
    Percorso:     CURRICULUM COMUNE
    Codice:       1
    Coorte:       2023
    Dipartimento: DIPARTIMENTO DI FISICA

[367BB] ELEMENTI DI COMPUTAZIONE
    Percorso:     CURRICULUM COMUNE
    Codice:       1
    Coorte:       2024
    Dipartimento: DIPARTIMENTO DI FISICA

[125ZW] PROVA FINALE
    Percorso:     FISICA TEORICA
    Codice:       1
    Coorte:       2023
    Dipartimento: DIPARTIMENTO DI FISICA

[125ZW] PROVA FINALE
    Percorso:     INTERAZIONI FONDAMENTALI
    Codice:       1
    Coorte:       2023
    Dipartimento: DIPARTIMENTO DI FISICA

[125ZW] PROVA FINALE
    Percorso:     FISICA DELLA MATERIA
    Codice:       1
    Coorte:       2023
    Dipartimento: DIPARTIMENTO DI FISICA

[125ZW] PROVA FINALE
    Percorso:     FISICA MEDICA
    Codice:       1
    Coorte:       2023
    Dipartimento: DIPARTIMENTO DI FISICA

[125ZW] PROVA FINALE
    Percorso:     ASTRONOMIA E ASTROFISICA
    Codice:       1
    Coorte:       2023
    Dipartimento: DIPAR

Sempre dalle pagine di search abbiamo il form di ricerca dei corsi di studio. Meno utile (tanto sono davvero pochi), ma tanto vale citare l'endpoing

`https://unipi.coursecatalogue.cineca.it/api/v1/ricercaCorsi`

in questo caso per esempio con payload

```json
{
    "searchString": "fisica",
    "anno": "2024",
    "tipoCorso": null,
    "sede": null,
    "lingua": null,
    "dipartimento": null,
    "interateneo": null,
    "area": null
}
```

Nello script a seguire il risultato della richiesta. La cosa davvero folle è la quantità colossale di informazioni che viene fornita da questa chiamata API. Non solo contiene una descrizione dei corsi ma perfino di tutti gli insegnamenti!!

In [55]:
# Interrogazione dell'API per ottenere informazioni sui corsi di studio

import requests
import yaml

targetFile = "searchCorsi.yaml"
url = "https://unipi.coursecatalogue.cineca.it/api/v1/ricercaCorsi"
payload = {
    "searchString": "fisica",
    "anno": "2024",
    "tipoCorso": None,
    "sede": None,
    "lingua": None,
    "dipartimento": None,
    "interateneo": None,
    "area": None
}
headers = {"Content-Type": "application/json"}
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    risultati = response.json()
    print(risultati)
    with open(targetFile, "w", encoding="utf-8") as f:
        yaml.dump(risultati, f, allow_unicode=True, sort_keys=False)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)


[{'cod': '1709634589846', 'des_it': 'Triennali e magistrali a ciclo unico', 'des_en': 'Degree Programme', 'note_it': '', 'note_en': '', 'subgroups': [{'cod': 'AR_SCI', 'des_it': 'Scienze matematiche, fisiche e della natura', 'des_en': 'Mathematical Physical and Natural Sciences', 'cds': [{'_id': '680480e0c337ce7fb7976d85', 'gdaId': 10441, 'ugovId': None, 'extCode': 'FIS-L', 'ate_cod': 'unipi', 'aa': '2024', 'cod': '10441', 'cdsId': '10441', 'cdsCod': 'FIS-L', 'des_it': 'FISICA', 'des_en': 'PHYSICS', 'lingua_cod': 'ita', 'lingua_des_it': 'Italiano', 'lingua_des_en': 'Italian', 'area_cod': 'AR_SCI', 'area_des_it': 'Scienze matematiche, fisiche e della natura', 'area_des_en': 'Mathematical Physical and Natural Sciences', 'sede_cod': '1059', 'sede_des_it': 'Università di Pisa', 'sede_des_en': 'Università di Pisa', 'sede_utenza_sostenibile': '186', 'altreSedi': [], 'crediti_it': '180', 'crediti_en': '180', 'durata_it': '3 anni', 'durata_en': '3 years', 'normativa_cod': 'DM270', 'normativa_i

Guardando le chiamate della pagina del CdL, sembra sia possibile accedere alle info del corso anche direttamente con una chiamata GET con il seguente endpoint

`https://unipi.coursecatalogue.cineca.it/api/v1/corso/2024/10452?annoOrdinamento=2023`

dove il parametro sull'ordinamento è irrilevante. In realtà basta indicare l'anno di erogazione (2024) e il codice del corso di studio. Ci sono codici rilevanti qui

* WFI-LM, Laurea Magistrale in Fisica dell'Università di Pisa: `10452`
* FIS-L, Laurea Triennale in Fisica dell'Università di Pisa: `10441`

In [59]:
# Esempio: come interrogare Course Catalogue per avere i dettagli di un corso di studi

import requests
import yaml

targetFile = "infoCdS.yaml"
url = "https://unipi.coursecatalogue.cineca.it/api/v1/corso/2024/10452"
response = requests.get(url)

if response.status_code == 200:

    data = response.json()
    print(f"Nome del CdS:        {data['des_it']}")
    print(f"Codice CdS:          {data['cdsCod']}")
    print(f"Id CdS (codice GDA): {data['cdsId']}")
    print(f"Anno accademico:     {data['aa']}")
    print(f"Ordinamento:         {data['ordinamento_aa']}")
    
    with open(targetFile, "w", encoding="utf-8") as file_yaml:
        yaml.dump(data, file_yaml, allow_unicode=True, sort_keys=False)
    
else:
    print(f"Errore nella richiesta: {response.status_code}")

Nome del CdS:        FISICA
Codice CdS:          WFI-LM
Id CdS (codice GDA): 10452
Anno accademico:     2024
Ordinamento:         2023


## Analisi JSON di interrogazione del CdS

Questo file di informazioni è particolarmente interessante e completo. Il file contiene

* Alcune informazioni di carattere generale (denominazione corso, anno accademico, eccetera)
* Un (breve) array di testi detti `requisiti_it` che descrive i requisiti di accesso al CdS, più versione inglese vuota.
* Un array di testi detti `programmi_testi_obiettivi_it` che descrivono contenuto, feedback, etc del CdS. Anche qui la versione inglese è sostanzialmente vuota.
* Un (breve) array `cariche_it` che indica i ruoli chiave, duplicato su `cariche_en`
* Un array di `strutture` che contiene solo il dipartimento... che è identificato dal codice `10184`
* Forse l'array più interessante è `percorsi`, che descrive i curricula ufficiali del CdS.

### Struttura dati dei Curricula

Ogni curriculum è identificato da un codice (univoco?), che cambia anche da un anno accademico all'altro. Ci sono alcune informazioni generali, per esempio fr quelle più interessanti...

```yaml
pdsId:      52570,     # <= codice univoco
pdsCod:     7,         # <= codice interno GDA
cdsId:      10452,     # <= riferimenti incrociato su CdS
schemaId:   8950,      # <= unclear... univoco non annuale?
schemaCod:  F TEOR,
des_it:     FISICA TEORICA,
```

Oltre a questo ogni curricular contiene gli insegnamenti organizzati per annualità `anni`. Per LM abbiamo primo e secondo anno. Ogni anno ha una indicazione `anno` (1 o 2), un anno di offerta e poi una lista di insegnamenti.

Gli insegnamenti sono a loro volta raggurppati. In questo caso speifico al primo anno abbiamo

- un codice `OO` di `Obbligatori`
- un codice `G` di `ASTROFISICA I ANNO F TEOR (6 CFU)`
- un codice `G` di `LABORATORI I ANNO F TEOR (9 CFU)`
- un codice `G` di `MICROFISICA I ANNO F TEOR (9 CFU)`
- un codice `G` di `LIBERI 15 CFU I ANNO F TEOR (15 CFU)`

al secondo anno invece

- un codice `OO` di `Obbligatori`... sostanzialemnte la prova finale.
- un codice `G` di `AFFINI E INTEGRATIVE II ANNO F TEOR (18 CFU)`

### Struttura dati del singolo insegnamento

I dati contengono un estratto delle informazioni chiave dell'insegnamento fra cui:

- `cod` contenente il codice univoco che compare anche nei link di Course Catalogue, esempio `52570_686735_71646`
- `adCod` è il classico codice interno dei corsi, tipo `214BB`
- `des-it` titolo in italiano
- `des-en` titolo in inglese
- `crediti` ... ovvio
- `ore` ... mi chiedo da dove arrivi questa informazione? Da GDA?
- `statoOfferta` questo è potenzialmente interessante perché potrebbe codificare lo stato di erogazione o meno, ma è tutto da verificare...
- `tafDes_it` vale `Caratterizzante` oppure  `Affine/Integrativa`
- `tipo_ins_des_it` a volte (molto raramente) indica `Necessaria Attivazione`, ma non è indicato quasi ma, è quasi un errore.
- `periodo_didattico_it` indica la semestralità
- (solo a volte! non affidabile) è indicato il `titolare` del corso... con dentro `matricola` e `des` ossia cognome nome.


internamente ci sono dei riferimenti incrociati, per esempio `corso_cod` conferma il codice del CdS, `corso_percorso_id` il codice del percorso (curriculum e così via) `ordinamento_aa` l'anno dell'ordinamneto di riferimento. Sembrano anche esistere alcuni link ad attività didattiche Padre e Figlie...



# Come affrontare la questione

La quantità di dati è grossa e le API forniscono informazioni in parte ridondanti, in parti inutili. Mappare tutto in funzione dell'anno di erogazione, del piano di studi, della coorte... sembra un lavoro assurdo anche con un approcico automatico